In [1]:
import pandas as pd

In [2]:
import re

In [3]:
import json
from typing import Union

In [4]:
json_path = "raw.json"

In [5]:
data_o = pd.read_json(json_path, orient="columns")

In [13]:
data = data_o.copy()

#### 1. Data Cleaning for "about" section : 
    As the section tell about description of the dataset, many of text analysis could be done on the mentioned column but as of now out motive is not looking into text analytics , hence we will replace the section with another column as "description_provided" as boolean True if description for a given Attraction place is provided or not.

In [14]:
def clean_about_section(dataframe, about_column_name) -> pd.DataFrame:
    # Where description is not present put False other wise its True
    dataframe["description_provided"] = dataframe[about_column_name].astype(bool)
    # There are no further necesity for the about column
    dataframe.drop(columns=[about_column_name], inplace=True)
    return dataframe

data = clean_about_section(data, "about")

#### Breadcrumbs

This column is actually act as a navigation to a particular geographical entity to the atttraction place. It holds very usefull insights regarding Country name, Location and State name.

In [15]:
def geographical_heirarchy(breadcrumbs: str) :
    if breadcrumbs:
        places = breadcrumbs.split(" > ")
        country = places[0]
        state = places[1]
        location = None
        state_region = None
        place_name_from_breadcrumbs = None
        separator_element = [places.index(l) for l in places if l.startswith('Places')]
        
        if separator_element:        
            location = places[separator_element[0]-1]
            state_region = places[separator_element[0]-2] if places[separator_element[0]-2]!= places[1] else None
            try :
                place_name_from_breadcrumbs = places[separator_element[0]+1]
            except : 
                place_name_from_breadcrumbs = None
        return country, state, state_region, location, place_name_from_breadcrumbs
    return None, None, None, None, None

x = data["breadcrumbs"].apply(geographical_heirarchy)
data = pd.concat([data, pd.DataFrame(x.to_list(), columns = ["country","state","state_region","location", "place_name_from_breadcrumbs"])],axis=1)
data.drop(columns=["breadcrumbs"], inplace=True)

#### Place Names 

There are instances where place names are not provided by scrapper, but we could get them from the breadcrumbs value

In [16]:
mask = data[data["place_name"].isnull()].index
data.loc[mask,"place_name"] = data.loc[mask,"place_name_from_breadcrumbs"]

# remove left over instances where place informatio is not provided
data = data[~data["place_name"].isnull()]

#### Suggested Durations : 
There is a mix of None and empty string present, thus we need to put them all None

In [18]:
def clean_suggested_duration(dataframe: pd.DataFrame, suggested_duration_column: str):
    dataframe.loc[dataframe[suggested_duration_column] == "", suggested_duration_column] = None
    return dataframe

data = clean_suggested_duration(data, "suggested_duration")

#### Ranking of a place based on its activity
For column "ranking_of_place" we can probably split it into the place rank, total number of places for such activity as well as activity names.

In [19]:
ranking_pattern= re.compile(r".+?([,\d]+)\s+\w+\s+([,\d]+)\s+(.*)\sin.*") 

def clean_ranking_of_place(text):
    ranking, total_activity_places, activity = None, None, text
    if isinstance(text, str) and text :
        match = ranking_pattern.match(text)
        if match:
            ranking = match.group(1).replace(",","")
            total_activity_places = match.group(2).replace(",","")
            activity = match.group(3)
    
    return ranking, total_activity_places, activity

# few values for ranking of place end up in place category due to scraping issues
mask = data[data["place_category"].fillna("").str.startswith("#")].index
data.loc[mask,"ranking_of_place"] = data.loc[mask,"place_category"]
data.loc[mask,'place_category'] = None

temp_split_data = data["ranking_of_place"].apply(clean_ranking_of_place)
data = pd.concat([data, pd.DataFrame(temp_split_data.to_list(), columns = ["ranking","total_activity_places","activity"]) ], axis=1)
data.drop(columns=["ranking_of_place"], inplace=True)

#### Place Category

This has some abnormal text characters present which should be taken out

In [20]:
def clean_place_category(text):
    if isinstance(text, str):
        text = re.sub(r"•",",", text)
        text = re.sub(r"\s+,\s+",", ",text)
        text = re.sub(r"\s+", " ", text)
    return text
    
data["place_category"] = data["place_category"].apply(lambda x : clean_place_category(x))

#### Nearby Restraunts witin 5kms
We hould remove all the text present in this column and only provide number of restraunt present within 5 km

In [21]:
# there are attractions present inside restraunt columns, hence we need to swap the values
mask = data[data["nearby_restraunts"].fillna("").str.endswith("10 kms")].index
data.loc[mask,"nearby_attractions"] = data.loc[mask,"nearby_restraunts"]
data.loc[mask,"nearby_restraunts"] = ""

def clean_nearby(text: str):
    if isinstance(text, str) and text:
        return text.strip().split()[0]
    return None

data["nearby_restraunts_within_5_kms"] = data["nearby_restraunts"].apply(clean_nearby)
data["nearby_attractions_within_10_kms"] = data["nearby_attractions"].apply(clean_nearby)

data.drop(columns=["nearby_restraunts","nearby_attractions"], inplace=True)

#### Total reviews count

These should be numbers but sometime there are text representations present

In [22]:
def clean_reviews(item:Union[str,int]) -> str:
    if isinstance(item, str):
        item = item.strip().split()[0].replace(",","")
        return item
    return 0

data["total_reviews_count"] = data["total_reviews_count"].apply(clean_reviews).astype(int)
data["excellent_reviews_count"] = data["excellent_reviews_count"].apply(clean_reviews).astype(int)
data["very_good_reviews_count"] = data["very_good_reviews_count"].apply(clean_reviews).astype(int)
data["average_reviews_count"] = data["average_reviews_count"].apply(clean_reviews).astype(int)
data["poor_reviews_count"] = data["poor_reviews_count"].apply(clean_reviews).astype(int)
data["terrible_reviews_count"] = data["terrible_reviews_count"].apply(clean_reviews).astype(int)

#### Average Revies

These must be present if not provided by the websites using the existing dataset

In [23]:
data["average_reviews"] = round(((5*data["excellent_reviews_count"] + 4*data["very_good_reviews_count"] + 3*data["average_reviews_count"] + 2*data["poor_reviews_count"] + 1*data["terrible_reviews_count"]) / (data["excellent_reviews_count"] + data["very_good_reviews_count"] + data["average_reviews_count"] + data["poor_reviews_count"] + data["terrible_reviews_count"]))*2)/2

#### Page URL 
This is an extra information , which has url for the page of extration scrapped for validation purpose, this has no such use for data analysis.

In [24]:
data.drop(columns = ["page_url"], inplace=True)

In [25]:
data = data[
    [
        "place_name",
        'country', 
        'state', 
        'state_region', 
        'location',
        "area",
        "place_category",
        "activity",
        'ranking',
        'total_activity_places',
        'total_reviews_count', 
        'excellent_reviews_count',
        'very_good_reviews_count', 
        'average_reviews_count',
        'poor_reviews_count', 
        'terrible_reviews_count',
        "average_reviews",
        "description_provided", 
        'nearby_restraunts_within_5_kms',
        'nearby_attractions_within_10_kms',
        "suggested_duration"
    ]
]

In [27]:
data.to_excel("cleaned.xlsx", index = False)

In [32]:
data.to_json("cleaned.json", orient = "table", indent = 2, index = False)